In [1]:
import pandas as pd
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
import scipy
import math
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq,df)
import statsmodels.api as sm

d:\python-installed\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
AirQualityCat = ['air-qual-good','air-qual-moderate','air-qual-unhealthy','air-qual-hazardous']
WeatherCat = ['weather-rainy','weather-stormy','weather-sunny','weather-cloudy','weather-hot','weather-cold','weather-dry','weather-wet','weather-windy','weather-snow']
TrafficCat = ['traffic-low','traffic-moderate','traffic-high','traffic-worse']
DayPartCat = ['dt-early-morning','dt-morning','dt-noon','dt-afternoon','dt-night','dt-late-night']
BikeLanesCat = ['bl-none','bl-partial','bl-full']
TemperatureCat = ['t-0-10','t-10-14','t-14-18','t-18-21','t-21-25','t-25-30','t-30-40','t-40-60','t-60-80','t-80-100']
ModesCat = ['mode-car','mode-transit','mode-bike','mode-mt-bike','mode-walk']

In [49]:
CatCombined = []
CatCombined.extend(AirQualityCat)
CatCombined.extend(WeatherCat)
CatCombined.extend(TrafficCat)
CatCombined.extend(DayPartCat)
CatCombined.extend(BikeLanesCat)
CatCombined.extend(TemperatureCat)

In [50]:
IndCombined = []
IndCombined.extend(CatCombined)
CatCombined.extend(ModesCat)

In [51]:

def ConvertTemperature(intValue,tempCategories):
    outArr = [0]*len(tempCategories)
    myVal = math.floor(intValue/len(tempCategories) - 1)
    outArr[myVal] = 1
    return outArr

def ConvertAirQuality(intValue,aqCategories):
    outArr = [0]*len(aqCategories)
    myVal = math.ceil((intValue / 2.5) - 1)
    outArr[myVal] = 1
    return outArr
    

In [59]:
xFile = pd.ExcelFile('data.xlsx')
df = xFile.parse('data')
dataArr = np.ones((len(IndCombined),len(ModesCat)))
for i in range(len(df.index)):
    freshVal = int(df.iloc[i]['Freshness'])
    freshVal = freshVal * 10
    tAirQual = ConvertAirQuality(int(df.iloc[i]['air-quality']),AirQualityCat)
    tWeather = df.iloc[i]['weather']
    tTraffic = df.iloc[i]['traffic']
    tDayTime = df.iloc[i]['day-time']
    tBikeLanes = df.iloc[i]['bike-lane']
    tTemperature = ConvertTemperature(int(df.iloc[i]['temperature']),TemperatureCat)
    tMode = df.iloc[i]['mode']
    tModeIndex = ModesCat.index(tMode);
    tAirQualIndex = tAirQual.index(1)
    tWeatherIndex = IndCombined.index(tWeather)
    tTrafficIndex = IndCombined.index(tTraffic)
    tDayTimeIndex = IndCombined.index(tDayTime)
    tBikeLanesIndex = IndCombined.index(tBikeLanes)
    
    dataArr[tAirQualIndex][tModeIndex] = freshVal + dataArr[tAirQualIndex][tModeIndex]
    dataArr[tWeatherIndex][tModeIndex] = freshVal + dataArr[tWeatherIndex][tModeIndex]
    dataArr[tTrafficIndex][tModeIndex] = freshVal + dataArr[tTrafficIndex][tModeIndex]
    dataArr[tDayTimeIndex][tModeIndex] = freshVal + dataArr[tDayTimeIndex][tModeIndex]
    dataArr[tBikeLanesIndex][tModeIndex] = freshVal + dataArr[tBikeLanesIndex][tModeIndex]
    dataArr[-len(TemperatureCat) + tTemperature.index(1)][tModeIndex] = freshVal + dataArr[-len(TemperatureCat) + tTemperature.index(1)][tModeIndex]
    
vert = [];
for i in range(len(IndCombined)):
    vert.extend([[IndCombined[i]]])
    
print(vert)
cr_tab = pd.crosstab(dataArr,index=vert,columns=ModesCat)


[['air-qual-good'], ['air-qual-moderate'], ['air-qual-unhealthy'], ['air-qual-hazardous'], ['weather-rainy'], ['weather-stormy'], ['weather-sunny'], ['weather-cloudy'], ['weather-hot'], ['weather-cold'], ['weather-dry'], ['weather-wet'], ['weather-windy'], ['weather-snow'], ['traffic-low'], ['traffic-moderate'], ['traffic-high'], ['traffic-worse'], ['dt-early-morning'], ['dt-morning'], ['dt-noon'], ['dt-afternoon'], ['dt-night'], ['dt-late-night'], ['bl-none'], ['bl-partial'], ['bl-full'], ['t-0-10'], ['t-10-14'], ['t-14-18'], ['t-18-21'], ['t-21-25'], ['t-25-30'], ['t-30-40'], ['t-40-60'], ['t-60-80'], ['t-80-100']]


TypeError: crosstab() got multiple values for argument 'index'

In [48]:
len(dataArr)

42

In [ ]:
def ConvertTemperature(intValue,tempCategories):
    outArr = [0]*len(tempCategories)
    myVal = math.floor(intValue/len(tempCategories))
    outArr[myVal] = 1
    return outArr

In [ ]:
ConvertTemperature(99,TemperatureCat)

In [ ]:
def ConvertAirQuality(intValue,aqCategories):
    outArr = [0]*len(aqCategories)
    myVal = math.ceil((intValue / 2.5) - 1)
    outArr[myVal] = 1
    return outArr

In [ ]:
ConvertAirQuality(6,AirQualityCat)